In [14]:

import numpy as np
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

def adaboost_train(X, y, T):
    """
    X: Dane treningowe, macierz NxM, gdzie N to liczba próbek, M to liczba cech.
    y: Etykiety klas, wektor o długości N.
    T: Liczba iteracji boostingu (liczba słabych klasyfikatorów).

    Zwraca:
    - klasyfikatory
    - ich wagi
    """
    N, M = X.shape
    # Inicjalizacja wag dla każdego przykładu (wszystkie wagi początkowo równe)
    w = np.ones(N) / N

    # Lista słabych klasyfikatorów
    classifiers = []
    # Lista wag dla każdego klasyfikatora
    alphas = []

    for t in range(T):
        # Tworzenie prostego klasyfikatora - tu użyjemy drzewo pniowe (ang. decision stump)
        stump = DecisionTreeClassifier(max_depth=1)

        # Trenowanie klasyfikatora na danych z uwzględnieniem wag
        stump.fit(X, y, sample_weight=w)
        # Predykcje klasyfikatora
        y_pred = stump.predict(X)

        # Obliczanie błędu klasyfikatora
        error = np.sum(w * (y_pred != y)) / np.sum(w)

        # Obliczanie wagi klasyfikatora
        alpha = 0.5 * np.log((1 - error) / (error + 1e-10))  # Dodajemy małą wartość, by uniknąć dzielenia przez zero

        # Aktualizacja wag dla przykładów
        w *= np.exp(-alpha * y * y_pred)
        w /= np.sum(w)  # Normalizacja wag, aby sumowały się do 1

        # Zapis klasyfikatora i jego wagi
        classifiers.append(stump)
        alphas.append(alpha)

    return classifiers, alphas





In [15]:

def adaboost_predict(X, classifiers, alphas):
    """
    X: Dane testowe
    classifiers: lista słabych klasyfikatorów
    alphas: lista wag klasyfikatorów

    Zwraca przewidywania dla danych X.
    """
    # Inicjalizacja przewidywań jako suma ważona klasyfikatorów
    final_predictions = np.zeros(X.shape[0])

    for classifier, alpha in zip(classifiers, alphas):
        # Zwiększanie wyniku o wagę razy predykcję danego klasyfikatora
        final_predictions += alpha * classifier.predict(X)

    # Finalna decyzja to znak sumy ważonej
    return np.sign(final_predictions)


In [16]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Załaduj dane Iris
data = load_iris()
X = data.data
y = data.target

In [17]:

# Wybierz tylko dwie klasy (dla uproszczenia)
X = X[y != 2]
y = y[y != 2]

# Zmiana etykiet klas na {-1, 1}
y = np.where(y == 0, -1, 1)

# Podziel dane na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [18]:

# Trenuj AdaBoost
T = 10  # Liczba słabych klasyfikatorów
classifiers, alphas = adaboost_train(X_train, y_train, T)

# Przewiduj etykiety na zbiorze testowym
y_pred = adaboost_predict(X_test, classifiers, alphas)

# Sprawdź dokładność
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność: {accuracy * 100:.2f}%")


Dokładność: 100.00%


In [31]:


def plot_decision_boundary(X, y, classifiers, alphas):
    # Ustawianie siatki punktów do wizualizacji
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))

    # Przewidywanie wartości dla każdego punktu siatki
    Z = adaboost_predict(np.c_[xx.ravel(), yy.ravel()], classifiers, alphas)
    Z = Z.reshape(xx.shape)

      # Rysowanie wykresu
    plt.contourf(xx, yy, Z, alpha=0.8 )
    plt.scatter(X[:, 0], X[:, 1], c=y )
    plt.show()